# CSPB 3104 Assignment 6/7

## Instructions

> This assignment is to be completed and uploaded to 
moodle as a python3 notebook. 

> Submission deadlines are posted on moodle. 

> The questions  provided  below will ask you to either write code or 
write answers in the form of markdown.

> Markdown syntax guide is here: [click here](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

> Using markdown you can typeset formulae using latex.

> This way you can write nice readable answers with formulae like thus:

>> The algorithm runs in time $\Theta\left(n^{2.1\log_2(\log_2( n \log^*(n)))}\right)$, 
wherein $\log^*(n)$ is the inverse _Ackerman_ function.

__Double click anywhere on this box to find out how your instructor typeset it. Press Shift+Enter to go back.__


## Question 1: Dynamic Programmer Jane's Progress

__Note:__ There is an accompanying set of images that should be placed in the same directory as this notebook.

We are writing a simple game AI for guiding our `Jane` the dynamic programmer to jump through a set of levels to reach a target level by taking
courses in dynamic programming.

The levels positions are numbered 1, ... , n. The character starts at level 1 and the goal is to reach level n (where she becomes
a d.p. ninja) and thus access CSCI 3104.
After taking a course, she can choose to move up by 1, 4, 5 or 11 levels forward at each step. No backward jumps are available.

![Jane_Programmer At Start of Game](jane-picture-p1.png "Jane at the Very Start of the Game" )

Your goal is to use dynamic programming to find out how to reach from level 1 to level n with the minimum number of courses.

## 1(A) Write a recurrence.

Write a recurrence `minCoursesForJane(j, n)` that represents the minimum number of steps for Jane to reach from level j to level n.


In [19]:
def minCoursesForJane(j, n):
    level_jumps = [1, 4, 5, 11]

    # base case j = n
    if j == n :
        return 0
    # base case j > n
    if j > n :
        return 999999999
    opts = [ 1 + minCoursesForJane(j + level_up, n) for level_up in level_jumps ]
    
    num_courses = min(opts)
    return num_courses

In [20]:
## Test Code: Do not edit
print(minCoursesForJane(1, 9)) # should be 2
print(minCoursesForJane(1, 13)) # should be 2
print(minCoursesForJane(1, 19)) # should be 4
print(minCoursesForJane(1, 34)) # should be 3
print(minCoursesForJane(1, 43)) # should be 5

2
2
4
3
5


## 1(B) Memoize the Recurrence.

Assume that n is fixed. The memo table $T[0], \ldots, T[n]$ should store the value of `minCoursesForJane(j, n)`. 

In [2]:
def minCoursesForJane_Memoize(n): 
    j = n - 1
    memo = [0] * (n + 1) # make the memo table with default values 0
    level_jumps = [1, 4, 5, 11]
    options = []

    # start at j = n - 1 and walk down to 1 to fill table
    # how many jumps (courses), does it take for Jane to 
    # get to level n from j? Each memo[j] answers that question
    # Jane starts at level 1, however so we return memo[1]
    while j >= 1 :
        options = [999999999]
        for jump in level_jumps :
            if j + jump <= n :
                options.append(memo[j + jump] + 1)
        memo[j] = min(options)
        j -= 1

    return memo[1]

In [3]:
## Test Code: Do not edit
print(minCoursesForJane_Memoize(9)) # should be 2
print(minCoursesForJane_Memoize(13)) # should be 2
print(minCoursesForJane_Memoize(19)) # should be 4
print(minCoursesForJane_Memoize(34)) # should be 3
print(minCoursesForJane_Memoize(43)) # should be 5

2
2
4
3
5


## 1(C) Recover the Solution

Modify the solution from part B to also return how many steps Jane needs to jump at each course.  Your answer must be
a pair: `minimum number of courses, list of jumps at each course: each elements of this list must be 1, 4, 5 or 11`


In [4]:
def minCoursesForJane_Solution(n): # Assume that j = 1 is always the starting point
   j = n - 1
   memo = [0] * (n + 1) # make the memo table with default values 0
   solution = [-1] * (n + 1) # make the solution table with default values -1
   level_jumps = [1, 4, 5, 11]
   jumps_made = []

   while j >= 1 :
      options = [(999999999, -1)]
      for jump in level_jumps :
         if j + jump <= n :
               options.append((memo[j + jump] + 1, jump))
      memo[j], solution[j] = min(options) # capture min pair for memo and solution table
      j -= 1

   # Now recover the jumps taken to reach n by using solution table
   curr_level = 1
   while curr_level < n :
      jumps_made.append(solution[curr_level])
      curr_level += solution[curr_level]
   assert curr_level == n

   return memo[1], jumps_made
    

In [5]:
## Test Code: Do not edit
print(minCoursesForJane_Solution(9)) # should be 2, [4, 4]
print(minCoursesForJane_Solution(13)) # should be 2, [1, 11]
print(minCoursesForJane_Solution(19)) # should be 4, [1, 1, 5, 11]
print(minCoursesForJane_Solution(34)) # should be 3, [11, 11, 11]
print(minCoursesForJane_Solution(43)) # should be 5, [4, 5, 11, 11, 11]

(2, [4, 4])
(2, [1, 11])
(4, [1, 1, 5, 11])
(3, [11, 11, 11])
(5, [4, 5, 11, 11, 11])


## 1(D) Greedy Solution

Suppose Jane tried a greedy strategy that works as follows. 
Initialize number of courses $c = 0$.

   1. While $n \geq 11$,
      1.1) jump $11$ steps forward, and set $n = n - 11$, $ c = c + 1$
   2. While $n \geq 5$, 
      2.1) jump $5$ steps forward and set $n = n - 5$, $ c = c + 1$
   3. While $n \geq 4$, 
      3.1) jump $4$ steps forward and set $n = n - 4$, $c = c + 1$
   4. Finally, while $n > 1$, 
      4.1) jump $1$ step forward and set $n = n - 1$, $c = c + 1$
     
This way, she can reach level $n$ starting from level $1$ using $c$ courses.

Show using an example for $n$ that this strategy may require her to take more courses than the optimal solution from dynamic programming.

## Answer (Expected Length 3 lines) 

I cannot help but notice this strategy is faulty since it leads to Jane ending up at level 0.  She is supposed to start from level 1 so this algorithm should lead to n = 1 not n = 0 for any of the following values: n = 4, 5, and 11, as well as many others.  I am going to assume the following structure as a result so that my starting place is found to be where it should be:

   1. While $n > 11$,
       jump $11$ steps forward, and set $n = n - 11$, $ c = c + 1$
   2. While $n > 5$, 
       jump $5$ steps forward and set $n = n - 5$, $ c = c + 1$
   3. While $n > 4$, 
       jump $4$ steps forward and set $n = n - 4$, $c = c + 1$
   4. Finally, while $n > 1$, 
       jump $1$ step forward and set $n = n - 1$, $c = c + 1$
    5. When n = 1, we have our solution.


We will start at level 1 but this algorithm starts at your final level and walks backwards.
- n = 9 = final level, c = 0
- n >= 5 but < 11    ->     n = 9 - 5 = 4,  c = 0 + 1 = 1
- n > 1 but < 4    ->     n = 4 - 1 = 3, c = 1 + 1 = 2
- n > 1 but < 4    ->     n = 3 - 1 = 2, c = 2 + 1 = 3
- n > 1 but < 4    ->     n = 2 - 1 = 1, c = 3 + 1 = 4
- n = 1.  c = 4

We can see from the first test code in part 1 that the ideal solution is making two 4 level jumps yielding an answer of 2 courses.
`print(minCoursesForJane_Solution(9)) # should be 2, [4, 4]`

----

## Question 2: The Defeat of Kilokahn

Unfortunately, life was not as simple as it seemed in problem 1. Some of the levels have been hacked by an evil group of 
students who can subvert Jane and her great expertise to serve evil Kilokahn (Kilometric Knowledge-base Animate Human Nullity). 

Any level j that leaves a remainder of 2 when divided by 7 is to be avoided by Jane as she progresses towards level n (where she
becomes a code ninja). However, Kilokahn will not be at level $n$ even if $n \mod 7 = 2$.


![Jane_Programmer At Start of Game with Kilokahn lurking](jane-picture-p2.png "Jane at the Very Start of the Game with Kilokahn lurking" )


## 2(A) Write a recurrence.

Write a recurrence `minCoursesForJaneAvoidKK(j, n)` that represents the minimum number of steps for Jane to reach from level j to level n while not reaching any level occupied by Kilokahn.


In [21]:
def minCoursesForJaneAvoidKK(j, n):
    level_jumps = [1, 4, 5, 11]
    options = []

    if j == n :
        return 0
    # base case j > n
    if j > n :
        return 999999999

    for jump in level_jumps :
        # special case -  j is congruent mod7 to 2 but j != n
        if (j + jump != n) and ((j + jump)%7 == 2) : # bad guy is here
             options.append(999999999)
        else :
            options.append(1 + minCoursesForJaneAvoidKK(j + jump, n))

    courses_taken = min(options)
    
    return courses_taken

In [23]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK(1, 9)) # should be 2
print(minCoursesForJaneAvoidKK(1, 13)) # should be 2
print(minCoursesForJaneAvoidKK(1, 19)) # should be 4
print(minCoursesForJaneAvoidKK(1, 34)) # should be 5
print(minCoursesForJaneAvoidKK(1, 43)) # should be 5
print(minCoursesForJaneAvoidKK(1, 55)) # should be 6 

2
2
4
5
5
6


## 2(B) Memoize the recurrence in 2(A)

In [86]:
def minCoursesForJaneAvoidKK_Memoize(n): # j is assumed to be 1 
    j = n - 1 # start at n - 1 since j = n takes 0 courses to get to
    memo = [0] * (n + 1) # make the memo table with default values 0
    level_jumps = [1, 4, 5, 11]
    options = []

    while j >= 1 :
        options = [999999999]
        for jump in level_jumps :
            if j + jump < n  and ((j + jump)%7 != 2) : # avoid the bad levels
                options.append(memo[j + jump] + 1)
            elif j + jump == n :
                options.append(memo[j + jump] + 1)
            else :
                options.append(999999999)
        memo[j] = min(options)
        j -= 1

    return memo[1]

In [87]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK_Memoize(9)) # should be 2
print(minCoursesForJaneAvoidKK_Memoize(13)) # should be 2
print(minCoursesForJaneAvoidKK_Memoize(19)) # should be 4
print(minCoursesForJaneAvoidKK_Memoize(34)) # should be 5
print(minCoursesForJaneAvoidKK_Memoize(43)) # should be 5
print(minCoursesForJaneAvoidKK_Memoize(55)) # should be 6
print(minCoursesForJaneAvoidKK_Memoize(69)) # should be 8
print(minCoursesForJaneAvoidKK_Memoize(812)) # should be 83

2
2
4
5
5
6
8
83


## 2(C) Recover the solution in terms of number of jumps for each course.

In [88]:
def minCoursesForJaneAvoidKK_Solution(n):
    j = n - 1
    memo = [0] * (n + 1) # make the memo table with default values 0
    solution = [-1] * (n + 1) # make the solution table with default values -1
    level_jumps = [1, 4, 5, 11]
    jumps_made = []

    while j >= 1 :
        options = [(999999999, -1)]
        for jump in level_jumps :
            if j + jump < n  and ((j + jump)%7 != 2) : # avoid the bad levels
                options.append((memo[j + jump] + 1, jump))
            elif j + jump == n :
                options.append((memo[j + jump] + 1, jump)) 
            else :
                options.append((999999999, jump))
            
        memo[j], solution[j] = min(options) # capture min pair for memo and solution table
        j -= 1

    # Now recover the jumps taken to reach n by using solution table
    curr_level = 1
    while curr_level < n :
        jumps_made.append(solution[curr_level])
        curr_level += solution[curr_level]
    assert curr_level == n

    return memo[1], jumps_made


In [89]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK_Solution(9)) # should be 2, [4, 4]
print(minCoursesForJaneAvoidKK_Solution(13)) # should be 2, [11, 1]
print(minCoursesForJaneAvoidKK_Solution(19)) # should be 4, [4, 5, 4, 5]
print(minCoursesForJaneAvoidKK_Solution(34)) # should be 5, [5, 1, 11, 11, 5]
print(minCoursesForJaneAvoidKK_Solution(43)) # should be 5, [4, 5, 11, 11, 11]
print(minCoursesForJaneAvoidKK_Solution(55)) # should be 6, [5, 11, 11, 11, 11, 5]
print(minCoursesForJaneAvoidKK_Solution(69)) # should be 8, [11, 1, 11, 11, 11, 11, 11, 1]
print(minCoursesForJaneAvoidKK_Solution(812)) # should be 83, [5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11]

(2, [4, 4])
(2, [11, 1])
(4, [4, 5, 4, 5])
(5, [5, 1, 11, 11, 5])
(5, [4, 5, 11, 11, 11])
(6, [5, 11, 11, 11, 11, 5])
(8, [11, 1, 11, 11, 11, 11, 11, 1])
(83, [5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11])


## Question 3: Energize Jane with a budget.

Unfortunately, life was not as simple as it seemed in problem 2. Besides dealing with Kilokahn, taking a course with a level jump consumes
a lot of Jane's energy, and she has an energy $E_0$ to begin with. Each time Jane jumps levels, she loses energy as follows:


| Jump   | Energy Consumed |
|--------|-----------------|
|  1     |       1         |
|  4     |       2         |
|  5     |       3         |
| 11     |       7         |


If at any point her energy level is $ \leq 0$ (even if she is at the destination), she will lose.

Given $n$, and initial energy $E_0$, plan how Jane can reach level $n$ (ninja level, in case you forgot) while
avoiding Kilokahn who  lurks when dividing the level by $7$ leaves a remainder of $2$ and keeping her energy levels
always strictly positive.

----

### 3(A): Write a Recurrence

Write a recurrence `minCoursesWithEnergyBudget(j, E, n)` that given that Jane is currently on level `j` with energy `E` finds the minimal 
number of courses she needs to take to reach `n`. Do not forget the base cases.

In [125]:
def minCoursesWithEnergyBudget(j, e, n):
    level_jumps = [1, 4, 5, 11]
    energy_consumed = [1, 2, 3, 7]
    options = []
    i = 0

    if j == n and e > 0 :
        return 0
    # base case j > n
    if e <= 0 or j > n :
        return 999999999

    for jump in level_jumps :
        # special case -  j is congruent mod7 to 2 but j != n
        if (j + jump != n) and ((j + jump)%7 == 2) : # bad guy is here
             options.append(999999999)
        else :
            options.append(1 + minCoursesWithEnergyBudget(j + jump, e - energy_consumed[i], n))
        i += 1

    courses_taken = min(options)
    
    return courses_taken

In [126]:
# test code do not edit
print(minCoursesWithEnergyBudget(1, 25, 10)) # must be 2
print(minCoursesWithEnergyBudget(1, 25, 6)) # must be 1
print(minCoursesWithEnergyBudget(1, 25, 30)) # must be 5
print(minCoursesWithEnergyBudget(1, 16, 30)) # must be 7
print(minCoursesWithEnergyBudget(1, 18, 31)) # must be 7
print(minCoursesWithEnergyBudget(1, 22, 38)) # must be 7
print(minCoursesWithEnergyBudget(1, 32, 55)) # must be 11
print(minCoursesWithEnergyBudget(1, 35, 60)) # must be 12

2
1
5
7
7
7
11
12


## 3(B): Memoize the Recurrence

Write a memo table to memoize the recurrence. Your memo table must be  of the form $T[j][e]$ for $j$ ranging from $1$ to $n$
and $e$ ranging from $0$ to $E$. You will have to handle the base cases carefully.

In [81]:
def minCoursesWithEnergyBudget_Memoize(e, n):
    level_jumps = [1, 4, 5, 11]
    energy_consumed = [1, 2, 3, 7]
    j = n - 1 # start at n - 1 since j = n takes 0 courses to get to
    k = e - 1 # our starting energy
    memo =  [[0]*(e+1)]*(n+1) # make the memo table with default values 0

    while j >= 1 :
        while k > 0 :
            options = [999999999]
            for i in range(len(level_jumps)) : # try each level
                if k + energy_consumed[i] > 0 : # go until energy is consumed
                    # make sure you don't hit bad levels
                    if j + level_jumps[i] < n  and ((j + level_jumps[i])%7 != 2) : # avoid the bad levels
                        options.append(memo[j + level_jumps[i]][k + energy_consumed[i]] + 1)
                    elif j + level_jumps[i] == n :
                        options.append(memo[j + level_jumps[i]][k + energy_consumed[i]] + 1)
                    else :
                        options.append(999999999)
                else :
                    options.append(999999999)
                
            memo[j][k] = min(options)
            k -= 1

        j -= 1

    return memo[1][1]

In [82]:
# test code do not edit
print(minCoursesWithEnergyBudget_Memoize(25, 10)) # must be 2
print(minCoursesWithEnergyBudget_Memoize(25, 6)) # must be 1
print(minCoursesWithEnergyBudget_Memoize(25, 30)) # must be 5
print(minCoursesWithEnergyBudget_Memoize(16, 30)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(18, 31)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(22, 38)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(32, 55)) # must be 11
print(minCoursesWithEnergyBudget_Memoize(35, 60)) # must be 12

24
24
24
15
17
21
31
34


## 3(C): Recover the Solution

Now write code that will also return the minimum number of courses along with the list of jumps that will achieve this minimum number

In [83]:
def minCoursesWithEnergyBudget_Solution(e, n):
    j = n - 1 # start at n - 1 since j = n takes 0 courses to get to
    memo = [0] * (n + 1) # make the memo table with default values 0
    solution = [-1] * (n + 1) # make the solution table with default values -1
    level_jumps = [1, 4, 5, 11]
    jumps_made = []

    #     while j >= 1 :
    #     options = [(999999999, -1)]
    #     for jump in level_jumps :
    #         if j + jump < n  and ((j + jump)%7 != 2) : # avoid the bad levels
    #             options.append((memo[j + jump] + 1, jump))
    #         elif j + jump == n :
    #             options.append((memo[j + jump] + 1, jump)) 
    #         else :
    #             options.append((999999999, jump))
            
    #     memo[j], solution[j] = min(options) # capture min pair for memo and solution table
    #     j -= 1

    # # Now recover the jumps taken to reach n by using solution table
    # curr_level = 1
    # while curr_level < n :
    #     jumps_made.append(solution[curr_level])
    #     curr_level += solution[curr_level]
    # assert curr_level == n

    # return memo[1], jumps_made

    while j >= 1 :
        options = [(999999999, -1)]
        for jump in level_jumps :
            if e >= 0 :
                if j + jump < n  and ((j + jump)%7 != 2) : # avoid the bad levels
                    options.append((memo[j + jump] + 1, jump))
                elif j + jump == n :
                    options.append((memo[j + jump] + 1, jump)) 
            else :
                options.append((999999999, jump))
        memo[j], solution[j] = min(options) # capture min pair for memo and solution table
        j -= 1

    # Now recover the jumps taken to reach n by using solution table
    curr_level = 1
    while curr_level < n :
        jumps_made.append(solution[curr_level])
        curr_level += solution[curr_level]
    assert curr_level == n

    return memo[1], jumps_made

In [84]:
# test code do not edit
print(minCoursesWithEnergyBudget_Solution(25, 10)) # must be 2, [4,5]
print(minCoursesWithEnergyBudget_Solution(25, 6)) # must be 1, [5]
print(minCoursesWithEnergyBudget_Solution(25, 30)) # must be 5, [4, 5, 4, 5, 11]
print(minCoursesWithEnergyBudget_Solution(16, 30)) # must be 7, [4, 5, 4, 4, 4, 4, 4]
print(minCoursesWithEnergyBudget_Solution(18, 31)) # must be 7, [4, 5, 4, 4, 4, 4, 5]
print(minCoursesWithEnergyBudget_Solution(22, 38)) # must be 7,  [4, 5, 4, 4, 4, 5, 11]
print(minCoursesWithEnergyBudget_Solution(32, 55)) # must be 11, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5]
print(minCoursesWithEnergyBudget_Solution(35, 60)) # must be 12, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5, 5]

(2, [4, 5])
(1, [5])
(5, [4, 5, 4, 5, 11])
(5, [4, 5, 4, 5, 11])
(5, [4, 5, 5, 5, 11])
(5, [5, 5, 11, 5, 11])
(6, [5, 11, 11, 11, 11, 5])
(7, [5, 5, 11, 5, 11, 11, 11])


----

## Question 4: Subset Sum Problem

We are given a set of whole numbers $S:\ \{ n_1, \ldots, n_k \}$ and a number $N$.
Our goal is to choose a subset of numbers $T:\ \{ n_{i_1}, \ldots, n_{i_j} \} \subseteq S$ such that

   (a) $\sum_{l=1}^j n_{i_l}  \leq N$, the sum of chosen numbers is less than or equal to $N$, 

   (b) The difference $N - \sum_{l=1}^j n_{i_l} $ is made as small as possible.

 For example, $S = \{ 1, 2, 3, 4, 5, 10 \}$ and $N = 20$ then by choosing $T = \{1, 2, 3, 4, 5\}$, we have  
$1 + 2 + 3 + 4 + 5 = 15 \leq 20$, achieving a difference of $5$. However, if we chose $T = \{ 2,3,5,10\}$ 
we obtain a sum of $2 + 3 + 5 + 10 = 20$ achieving the smallest possible difference of $0$.


Therefore the problem is as follows:

  * Inputs: list  $S: [n_1, \ldots, n_k]$ and number $N$.
  * Output: a list $T$ of elements from $S$ such that sum of elements of $T$ is  $\leq N$ and $N - \sum_{e \in T} e$ is the smallest possible.

The subsequent parts to this problem ask you to derive a dynamic programming solution to this problem.

__Note:__ Because $S$ and $T$ are viewed as sets, each element in the set may occur exactly once.

 ## 4(A) Show how the decisions can be staged to obtain optimal substructure (expected size: 5 lines)

S = [n1, n2, n3, ..., nk]

$$\text{minDiffSet}(S, N) = \min\ \left\{ \begin{array}{ll}
N & \leftarrow\ \text{Choose no value and quit}\\
N - \sum [n_1, \text{minDiffSet}(S - n_1, N - n_1)] & \leftarrow\ \text{Choose n1 number for sum}\  \\
N - \sum [n_2, \text{minDiffSet}(S - n_2, N - n_2)] & \leftarrow\ \text{Choose n2 number for sum}\  \\
\vdots & \\
N - \sum [n_k, \text{minDiffSet}(S - n_k, N - n_{k})] & \leftarrow\ \text{Choose nk number for sum}\ \\
\end{array} \right.$$

Base Case:

  * $\text{minDiffSet}(S, 0) = [ \text{empty set} ]$ 
  * $\text{minDiffSet}(S, N) = \infty$ if $N < 0$.
 
 


## 4(B): Write a recursive function for calculating the minimum value of the difference possible. 

In [125]:
def minSubsetDifference_recursive(N, s_list):
    min_diff = N
    differences = [999999999]

    if N == 0 :
        return 0
    elif N < 0 :
        return 999999999
    
    for i in range(len(s_list)) :
        if N - s_list[i] > 0 :
            lst_sofar = s_list[:]
            lst_sofar.pop(i)
            differences.append( minSubsetDifference_recursive(N - s_list[i], lst_sofar[:]))
        else :
            differences.append(999999999)
    min_diff = min(differences)
    return min_diff

In [126]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference_recursive(15, [1, 2, 3, 4, 5, 10])) # Should be zero
print(minSubsetDifference_recursive(26, [1, 2, 3, 4, 5, 10])) # should be 1
print(minSubsetDifference_recursive(23, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(18, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(9, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1
print(minSubsetDifference_recursive(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0
print(minSubsetDifference_recursive(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1

999999999
999999999
999999999
999999999
999999999
999999999
999999999
999999999


## 4(C): Memoize the recurrence above. 

To help with your memoization, use a 2D memo table  $T[n][j]$ that represents the value for `minSubsetDifference(n, s_list[0:j])`. 

In [127]:
def minSubsetDifference_Memoize(N, s_list):
    return 129

In [128]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference_Memoize(15, [1, 2, 3, 4, 5, 10])) # Should be 0
print(minSubsetDifference_Memoize(26, [1, 2, 3, 4, 5, 10])) # should be 1
print(minSubsetDifference_Memoize(23, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(18, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(9, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1
print(minSubsetDifference_Memoize(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0
print(minSubsetDifference_Memoize(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1

129
129
129
129
129
129
129
129


## 4(D): Write code to recover the solution

In [129]:
def minSubsetDifference(N, s_list):
    return 121, [1, 4, 6, 2, 1]

In [130]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference(15, [1, 2, 3, 4, 5, 10])) # Should be 0, [5, 4, 3, 2, 1]
print(minSubsetDifference(26, [1, 2, 3, 4, 5, 10])) # should be 1, [10, 5, 4, 3, 2, 1]
print(minSubsetDifference(23, [1, 2, 3, 4, 5, 10])) # should be 0, [10, 5, 4, 3, 1]
print(minSubsetDifference(18, [1, 2, 3, 4, 5, 10])) # should be 0, [10, 4, 3, 1]
print(minSubsetDifference(9, [1, 2, 3, 4, 5, 10])) # should be 0, [4, 3, 2]
print(minSubsetDifference(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1, [339, 94, 23]
print(minSubsetDifference(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0, [312, 152, 37, 11]
print(minSubsetDifference(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1, [413, 94, 48, 37]

(121, [1, 4, 6, 2, 1])
(121, [1, 4, 6, 2, 1])
(121, [1, 4, 6, 2, 1])
(121, [1, 4, 6, 2, 1])
(121, [1, 4, 6, 2, 1])
(121, [1, 4, 6, 2, 1])
(121, [1, 4, 6, 2, 1])
(121, [1, 4, 6, 2, 1])


## 4 (E): Greedy Solution

Suppose we use the following greedy solution to solve the problem.
  * $T = \emptyset$
  * While ( $ N \geq 0 $) 
    * Select the largest element $e$ for $S$ that is smaller than $N$
    * Remove $e$ from $S$
    * Add $e$ to $T$
    * N = N - e
  * return (N, T)
  
Using an example, show that the greedy algorithm does not necessarily produce the optimal solution.

### Answer (4 lines)

S = [1, 4, 5, 6, 10]
N = 9

If we choose 6 with the greedy algorithm, we are left with 9 - 6 = 3.  We can only choose 1 to make up the difference yielding a difference of 9 - 6 - 1 = 2.
The ideal solution however is choosing elements [4, 5], creating a difference of zero, 9 - 5 - 4 = 0.

## Testing your solutions -- Do not edit code beyond this point